In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import time
import os
#from google.colab.patches import cv2_imshow
print(cv2.__version__)

4.5.3


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras

In [3]:
def vgg13_model(n_classes):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), input_shape=(64, 64, 1), padding='same', activation='relu',
                     kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [4]:
model = vgg13_model(8)
model.load_weights('./model_size64_Adagrad/model_40-0.80.hdf5')

In [6]:
video = cv2.VideoCapture("test.mp4")
if not video.isOpened():
    print("Could not open video")

# Read first frame.
ok, frame = video.read()
if not ok:
    print('Cannot read video file')
    
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "neutral", 1: "happiness", 2: "surprise", 3: "sadness", 4: "anger", 5: "disgust", 6: "fear", 7: "contempt"}

while True:
    # Find haar cascade to draw bounding box around face
    ok, frame = video.read()
    if not ok:
        break

    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    c = 0
    b = 0
    frame = cv2.addWeighted(frame, c, frame, 1-c, b)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.equalizeHist(gray, gray)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.15, minNeighbors=5, minSize=(5, 5),
        flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img=cv2.resize(roi_gray,(64,64), interpolation=cv2.INTER_CUBIC)
        input_data=np.array(cropped_img)
        input_data = input_data.reshape(-1, 64, 64, 1)
        prediction = model.predict(input_data)
        #print(prediction)
        maxindex1 = int(np.argmax(prediction))
        promax1 = prediction[0,maxindex1]
        prediction[0,maxindex1] = 0.0
        maxindex2 = int(np.argmax(prediction))
        promax2 = prediction[0,maxindex2]
        #print(maxindex)
        #if(maxindex1 == 0 and promax2 > 0.1):
        #    maxindex = maxindex2
        #else:
        #    maxindex = maxindex1
        #cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxindex1] + "{:.2f}".format(promax1), (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxindex2] + "{:.2f}".format(promax2), (x+40, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(704,576),interpolation = cv2.INTER_CUBIC))
    #cv2.imshow('Video', cv2.resize(gray,(704,576),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
tracker = cv2.TrackerKCF_create()
video = cv2.VideoCapture("test.mp4")

if not video.isOpened():
    print("Could not open video")

# Read first frame.
ok, frame = video.read()
if not ok:
    print('Cannot read video file')

# Local computer, use mouse to select the target bounding box, press ESC to finish
bbox = cv2.selectROI(frame, False)
#bbox =  tuple([370, 260, 50, 50])
ok = tracker.init(frame, bbox)

frameIndex = 0
frameTotal = 10
while (1): # Specify the total number of frames displayed in Colab
    frameIndex+=1
    # Read a new frame
    ok, frame = video.read()
    if not ok:
        break

    # Update tracker
    ok, bbox = tracker.update(frame)
    # Draw bounding box
    if ok:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        roi_gray = gray[int(bbox[1]):int(bbox[1] + bbox[3]),int(bbox[0]):int(bbox[0] + bbox[2])]
        cropped_img=cv2.resize(roi_gray,(64,64), interpolation=cv2.INTER_LINEAR)
        cv2.imshow("t", cropped_img)
        input_data=np.array(cropped_img)
        input_data = input_data.reshape(-1, 64, 64, 1)
        prediction = model.predict(input_data)
        maxindex1 = int(np.argmax(prediction))
        promax1 = prediction[0,maxindex1]
        prediction[0,maxindex1] = 0.0
        maxindex2 = int(np.argmax(prediction))
        promax2 = prediction[0,maxindex2]
        cv2.putText(frame, emotion_dict[maxindex1] + "{:.2f}".format(promax1), (0, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxindex2] + "{:.2f}".format(promax2), (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display tracker type on frame
    cv2.putText(frame, "Tracker, Frame " + str(frameIndex), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display result
    cv2.imshow("Tracking", frame) # local computer
    #cv2_imshow(frame) # colab

    # Exit if ESC pressed (for local computer)
    k = cv2.waitKey(0) & 0xff
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()

Could not open video
Cannot read video file
